In [1]:
import os
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
import string

In [2]:
data_dir = '../input/flickr-image-dataset/flickr30k_images'
img_dir = os.path.join(data_dir, 'flickr30k_images')
images = list(os.listdir(img_dir))

data_file = os.path.join(data_dir, 'results.csv')


In [3]:
data = pd.read_csv(data_file, delimiter = '|')
data.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [4]:
data.columns = ['image_name','comment_number','comment']

In [5]:
print ("Number of images in image forlder = %d and number of entries in results.csv = %d" %(len(images), data.shape[0]))


Number of images in image forlder = 31785 and number of entries in results.csv = 158915


Notice that for one image there can be multiple labels in the comment section. For example, for image id 1000092795.jpg, there are 5 comments denoted by the comment number which describes that image. 

In [6]:
data[data['image_name']=='1000092795.jpg']

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [7]:
#finding the number of unique image ids from results
len(np.unique(data['image_name'].values))

31783

So, there are two extra images in image folder for which there is no description given in the data. We are not going to consider these two images for our model building. Further for each unique image there are 5 separate captions possible. So, 31783*5 = 158915 entries of the dataframe.

### Building a map from image id to various comments

In [8]:
from tqdm import tqdm
image2comment = {}
unique_images = np.unique(data['image_name'].values)

for img_id in tqdm(unique_images):
    comments = list(data[data['image_name']==img_id]['comment'].values)
    image2comment[img_id] = comments

100%|██████████| 31783/31783 [13:08<00:00, 40.32it/s]


## Preprocessing the language data/comments

In [9]:
table = str.maketrans('', '', string.punctuation)
def preprocess(comment):
    #1) tokenize
    tokens = word_tokenize(comment)
    
    #2) convert tokens to lower case
    tokens = [token.lower() for token in tokens]
    
    #3) Remove punctuations from tokens
    tokens = [token.translate(table) for token in tokens]
    
    #4) Remove tokens which has a length less than 1
    tokens = [token for token in tokens if len(token) >1]
    
    #5) Only consider tokens which are pure words(only strings of alphabets)
    tokens = [token for token in tokens if token.isalpha()]
    
    preprocessed_comment = ' '.join(tokens)
    
    return preprocessed_comment
    

In [10]:
comment = data['comment'].values[110]
print (comment)
preprocess(comment)

 A man with a goatee in a black shirt and white latex gloves is using a tattoo gun to place a tattoo on someone 's back .


'man with goatee in black shirt and white latex gloves is using tattoo gun to place tattoo on someone back'

In [11]:
#preprocessing on all comments
preprocessed_img_comments = {}
comment_count = 0
for img_id in tqdm(image2comment.keys()):
    comments = image2comment[img_id]
    t = []
    for comment in comments:
        comment_count +=1
        try:
            t.append(preprocess(comment))
        except:
            print (img_id,comment, comment_count)
            
    preprocessed_img_comments[img_id] = t
        

 13%|█▎        | 4181/31783 [00:04<00:28, 977.85it/s]

2199200615.jpg nan 20000


100%|██████████| 31783/31783 [00:32<00:00, 963.58it/s]


In [12]:
data[data['image_name'] =='2199200615.jpg']['comment'].values

array([' A dog running on green grass with its mouth open and tongue hanging out .',
       ' a white dog is running with its mouth open across the grass .',
       ' A white , black , and brown dog runs in a field .',
       ' A dog runs across the grassy field .', nan], dtype=object)

Notice that this image id has 4 comments instead of 5,i.e, one comment is ''. A try except is used as a wrapper while preprocessing the comment so as to remove the empty comment. The preprocessed data now should have (158915 -1) = 158914  comments.

In [13]:
s = 0
for img_id in preprocessed_img_comments.keys():
    s += len(preprocessed_img_comments[img_id])
print (s)

158914


## Splitting data into train and test sets

Splitting data into 80% train data and 20% test data.

In [14]:
from sklearn.model_selection import train_test_split
train_ids, test_ids = train_test_split(unique_images, test_size = 0.2, random_state = 42, shuffle = True)

print (len(train_ids), len(test_ids))

25426 6357


In [15]:
train_data = {}
for train_idx in train_ids:
    train_data[train_idx] = preprocessed_img_comments[train_idx]
    
    
test_data = {}
for test_idx in test_ids:
    test_data[test_idx] = preprocessed_img_comments[test_idx]

In [16]:
len(train_data.keys()), len(test_data.keys())

(25426, 6357)

# Working on train set

First, I am going to further divide the train set into train and train and validation sets with validation sets containing 20% of the train set data.

In [17]:
train_ids, valid_ids = train_test_split(list(train_data.keys()), shuffle = True, random_state = 42, test_size = 0.2)
train = {}
valid = {}

for id in train_ids:
    train[id] = train_data[id]
    
for id in valid_ids:
    valid[id] = train_data[id]
    
print (len(train.keys()), len(valid.keys()))

20340 5086


## Building vocabulary

For train set, I will select all the unique words and add it to the vocabulary. There will be another two extra tokens a) out of vocabulary token (<unk>) and pad token(<pad>).

In [18]:
vocab = {}
for id in train.keys():
    comments = train[id]
    for comment in comments:
        words = comment.split(' ')
        for word in words:
            vocab[word] = vocab.get(word, 0) + 1
                
print ("Number of unique words in train corpus = %d" %(len(vocab.keys())))

Number of unique words in train corpus = 16506


Since the vocabulary dimension is pretty high, I am going to consider only those words which has atleast more than 30 occurence over the train corpus.

In [19]:
words = [word for word in vocab.keys() if vocab[word]>30]
len(words)

2197

In [20]:
vocab_words = set(words)
vocab_words.add('<unk>')
vocab_words.add('<pad>')

print (len(vocab_words))

2199
